In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time
import os

def click_more_button(driver, max_clicks):
    for _ in range(max_clicks):
        try:
            # '더보기' 버튼 찾기 (class와 data-page 속성 활용)
            more_button = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "a.button.expanded.nd-white.list-btn-more"))
            )
            
            # 버튼 클릭
            driver.execute_script("arguments[0].click();", more_button)
            
            print("더보기 버튼 클릭 (JavaScript 실행)")
            time.sleep(2)  # 데이터 로딩 대기
            
        except Exception as e:
            print(f"더보기 버튼 클릭 실패: {e}")
            break

def get_news(driver):
    # 기사 크롤링
    fieldnames = ["title", "thumbnail", "short_content", "write_time", "link", "company"]
    data = []

    
    # 페이지 열기
    url = f'https://www.joongang.co.kr/trend'
    driver.get(url)
    time.sleep(2)

    click_more_button(driver, max_clicks=3)

    articles = driver.find_elements(By.CSS_SELECTOR, '.story_list.story_rank > li')

    for article in articles:
        try:
            # 제목(title)
            title_element = article.find_element(By.CSS_SELECTOR, '.headline > a')
            title = title_element.text.strip()

            # 썸네일(thumbnail)
            thumbnail_element = article.find_elements(By.CSS_SELECTOR, '.card_image > a > img')
            thumbnail = thumbnail_element[0].get_attribute('src').strip() if thumbnail_element else None

            # 요약 내용(short_content) 없음 
            short_content_element = article.find_elements(By.CSS_SELECTOR, '.aabbcc')
            short_content = short_content_element[0].text.strip() if short_content_element else None
            
            # 작성 시간(write_time)
            write_time_element = article.find_elements(By.CSS_SELECTOR, '.date')
            write_time = write_time_element[0].text.strip() if write_time_element else None

            # 기사 링크(link) - **href**로 수정
            link = title_element.get_attribute('href').strip()

            # 데이터 추가
            data.append({
                "title": title,
                "thumbnail": thumbnail,
                "short_content": short_content,
                "write_time": write_time,
                "link": link,
                "company": "중앙일보"
            })

        except Exception as e:
            print(f"데이터 수집 중 오류 발생: {e}")
            continue

    # CSV 파일로 저장
    save_to_csv(data, fieldnames)

    driver.quit()

def save_to_csv(data, fieldnames):
    # CSV 파일 저장
    folder_path = './Completed_csv'
    
    # 폴더가 없으면 생성
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"{folder_path} 폴더가 생성되었습니다.")

    # 파일 경로 설정
    filename = os.path.join(folder_path, 'JoongAng.csv')
    
    # CSV 파일 저장
    try:
        with open(filename, mode='w', newline='', encoding='utf-8-sig') as file:
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()  # 헤더 작성
            for item in data:
                writer.writerow(item)
        print(f"데이터가 저장되었습니다.")
    except Exception as e:
        print(f"CSV 저장 중 오류 발생: {e}")

from selenium import webdriver
from selenium.webdriver.chrome.service import Service

def main():
    options = Options()
    options.add_argument('--headless')  # GUI 없이 실행
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    # WebDriverManager를 사용하여 ChromeDriver 자동 관리
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    print("ChromeDriver 실행 완료")

    get_news(driver)

    driver.quit()

if __name__ == '__main__':
    main()


C:\Users\이형석\AppData\Local\Programs\Python\Python36\lib\site-packages\requests\__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.20) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  RequestsDependencyWarning)
[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 133.0.6943
[WDM] - Get LATEST chromedriver version for 133.0.6943 google-chrome


ValueError: There is no such driver by url https://chromedriver.storage.googleapis.com/LATEST_RELEASE_133.0.6943